# Rendimento escolar nos municípios do Estado de São Paulo

#### Base: Taxas de Rendimento Escolar – INEP

**Como evoluíram as taxas de aprovação, reprovação e abandono nos municípios de São Paulo nos últimos anos, e como as diferenças entre redes de ensino e etapas indicam disparidades educacionais dentro do estado?**

**Importando bibliotecas**

In [26]:
# Manipulação de dados grandes
import dask.dataframe as dd

# Manipulação leve/auxiliar
import pandas as pd
import numpy as np

# Visualizações
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

# Configurações estéticas
sns.set(style="whitegrid")
plt.rcParams["figure.figsize"] = (10, 6)

# Geoespacial 
import geopandas as gpd

# Sistema
import os


In [27]:
file_path = "data/tx_rend_municipios_2024.xlsx"

df = pd.read_excel(file_path, skiprows=8)

ddf = dd.from_pandas(df, npartitions=4)


In [28]:
print("shape:", ddf.shape)
display(ddf.head(3))
display(ddf.columns.tolist())

shape: (<dask_expr.expr.Scalar: expr=df.size() // 61, dtype=int64>, 61)


,NU_ANO_CENSO,NO_REGIAO,SG_UF,CO_MUNICIPIO,NO_MUNICIPIO,NO_CATEGORIA,NO_DEPENDENCIA,1_CAT_FUN,1_CAT_FUN_AI,1_CAT_FUN_AF,...,3_CAT_FUN_06,3_CAT_FUN_07,3_CAT_FUN_08,3_CAT_FUN_09,3_CAT_MED,3_CAT_MED_01,3_CAT_MED_02,3_CAT_MED_03,3_CAT_MED_04,3_CAT_MED_NS
0,2024,Norte,RO,1100015.0,Alta Floresta D'Oeste,Total,Total,96.8,97.1,96.5,...,0.3,0.3,0,0.3,0.4,0.5,0.5,0,--,--
1,2024,Norte,RO,1100015.0,Alta Floresta D'Oeste,Urbana,Total,97.1,96.8,97.4,...,0.4,0.4,0,0.4,0.5,0.6,0.6,0,--,--
2,2024,Norte,RO,1100015.0,Alta Floresta D'Oeste,Rural,Total,96.3,97.6,94.9,...,0,0,0,0,0,0,0,0,--,--


['NU_ANO_CENSO',
 'NO_REGIAO',
 'SG_UF',
 'CO_MUNICIPIO',
 'NO_MUNICIPIO',
 'NO_CATEGORIA',
 'NO_DEPENDENCIA',
 '1_CAT_FUN',
 '1_CAT_FUN_AI',
 '1_CAT_FUN_AF',
 '1_CAT_FUN_01',
 '1_CAT_FUN_02',
 '1_CAT_FUN_03',
 '1_CAT_FUN_04',
 '1_CAT_FUN_05',
 '1_CAT_FUN_06',
 '1_CAT_FUN_07',
 '1_CAT_FUN_08',
 '1_CAT_FUN_09',
 '1_CAT_MED',
 '1_CAT_MED_01',
 '1_CAT_MED_02',
 '1_CAT_MED_03',
 '1_CAT_MED_04',
 '1_CAT_MED_NS',
 '2_CAT_FUN',
 '2_CAT_FUN_AI',
 '2_CAT_FUN_AF',
 '2_CAT_FUN_01',
 '2_CAT_FUN_02',
 '2_CAT_FUN_03',
 '2_CAT_FUN_04',
 '2_CAT_FUN_05',
 '2_CAT_FUN_06',
 '2_CAT_FUN_07',
 '2_CAT_FUN_08',
 '2_CAT_FUN_09',
 '2_CAT_MED',
 '2_CAT_MED_01',
 '2_CAT_MED_02',
 '2_CAT_MED_03',
 '2_CAT_MED_04',
 '2_CAT_MED_NS',
 '3_CAT_FUN',
 '3_CAT_FUN_AI',
 '3_CAT_FUN_AF',
 '3_CAT_FUN_01',
 '3_CAT_FUN_02',
 '3_CAT_FUN_03',
 '3_CAT_FUN_04',
 '3_CAT_FUN_05',
 '3_CAT_FUN_06',
 '3_CAT_FUN_07',
 '3_CAT_FUN_08',
 '3_CAT_FUN_09',
 '3_CAT_MED',
 '3_CAT_MED_01',
 '3_CAT_MED_02',
 '3_CAT_MED_03',
 '3_CAT_MED_04',
 '3_CA

A base original do INEP contém informações de todos os municípios do Brasil, para todas as regiões, estados, dependências administrativas e etapas de ensino.
Como o objetivo do projeto é analisar exclusivamente o estado de São Paulo, nosso primeiro passo é filtrar a coluna SG_UF para manter apenas as linhas onde o valor é "SP".

Isso garante que:

- evitamos processar dados desnecessários, deixando a análise mais leve;

- todas as visualizações e estatísticas se concentram apenas no contexto paulista, como definido na questão de pesquisa;

- eliminamos ruídos de municípios de outros estados.

A tabela contém mais de 60 colunas, mas nem todas são necessárias para este projeto. Vamos selecionar somente as colunas que têm informação útil para análise do rendimento escolar:

1. Informações de identificação do município

**CO_MUNICIPIO** — código IBGE do município (permite agregações consistentes)

**NO_MUNICIPIO** — nome do município

Essas colunas permitem identificar cada observação corretamente e comparar municípios dentro do estado.

2. Informações organizacionais

**NO_CATEGORIA** — categoria (ex.: Urbano / Total)

**NO_DEPENDENCIA** — tipo de rede administrativa
(ex.: Municipal, Estadual, Federal, Privada)

Essas variáveis são importantes para:

- comparar o desempenho entre redes (ex.: Estadual × Municipal)

- identificar diferenças entre áreas urbanas, rurais ou totais

- fazer análises estratificadas

3. Colunas de rendimento escolar (principais indicadores)

Essas colunas começam com:

**1_** → Taxas de aprovação

**2_** → Taxas de reprovação

**3_** → Taxas de abandono

E aparecem em diferentes recortes, como:

Total

Anos iniciais

Anos finais

Por série (1º ao 9º ano)

Ensino médio (1ª, 2ª, 3ª série)

Essas são as variáveis centrais do projeto — toda a análise de desempenho, rankings, tendências e comparações é feita usando esses valores.

Portanto, selecionamos todas as colunas que começam com 1_, 2_ ou 3_.

As colunas de taxas de rendimento escolar (todas que começam com 1_, 2_ e 3_) contêm valores numéricos representando percentuais, mas a base original do INEP inclui valores como:

- **"--"**

- **"."**

- **"0 "**

- valores com vírgula ("97,1")

- strings não numéricas

Esses formatos impedem que o Python trate as colunas como números reais.
Por isso, antes de iniciarmos qualquer análise, precisamos padronizar as taxas.

Nesta etapa, vamos:

1. Identificar automaticamente todas as colunas de taxa

(prefixos 1_, 2_, 3_ conforme definido pelo INEP)

2. Substituir valores especiais por valores nulos (NaN)

Isso é importante porque NaN não atrapalha cálculos estatísticos.

3. Converter todas as colunas de taxa para tipo numérico (float)

Aplicando:

- substituição de vírgula por ponto

- coerção segura (errors="coerce")

4. Validar a conversão, mostrando estatísticas descritivas

Assim confirmamos que os valores foram reconhecidos corretamente.

Essa limpeza garante que os próximos passos — como criar rankings, gráficos e mapas — funcionem sem erros.

In [38]:
# ---------------------------------------
# Filtrar SP com segurança e diagnóstico
# ---------------------------------------

# 1. Normalizar SG_UF para uppercase
ddf["SG_UF"] = ddf["SG_UF"].astype(str).str.strip().str.upper()

# 2. Filtrar SP (lazy)
ddf_sp = ddf[ddf["SG_UF"] == "SP"]

# 3. Diagnóstico - contar linhas
print("Contando linhas SP (compute)...")
n_sp = ddf_sp.shape[0].compute()
print("Número de linhas com SG_UF == 'SP':", n_sp)

# 4. Mostrar primeiras linhas (agora computadas)
print("\nPreview de SP:")
df_sp = ddf_sp.compute()
df_sp.head()


Contando linhas SP (compute)...
Número de linhas com SG_UF == 'SP': 6703

Preview de SP:


,NU_ANO_CENSO,NO_REGIAO,SG_UF,CO_MUNICIPIO,NO_MUNICIPIO,NO_CATEGORIA,NO_DEPENDENCIA,1_CAT_FUN,1_CAT_FUN_AI,1_CAT_FUN_AF,...,3_CAT_FUN_06,3_CAT_FUN_07,3_CAT_FUN_08,3_CAT_FUN_09,3_CAT_MED,3_CAT_MED_01,3_CAT_MED_02,3_CAT_MED_03,3_CAT_MED_04,3_CAT_MED_NS
40160,2024,Sudeste,SP,3500105.0,Adamantina,Total,Total,99.5,99.7,99.3,...,0.2,0.5,0,0.7,8.6,6,8.5,11.9,--,--
40161,2024,Sudeste,SP,3500105.0,Adamantina,Urbana,Total,99.5,99.7,99.3,...,0.2,0.5,0,0.7,9.4,6.7,8.9,12.7,--,--
40162,2024,Sudeste,SP,3500105.0,Adamantina,Rural,Total,--,--,--,...,--,--,--,--,0,0,0,0,--,--
40163,2024,Sudeste,SP,3500105.0,Adamantina,Total,Estadual,99.4,--,99.4,...,0.4,0.8,0,1.1,10.7,7.6,10.6,14.4,--,--
40164,2024,Sudeste,SP,3500105.0,Adamantina,Urbana,Estadual,99.4,--,99.4,...,0.4,0.8,0,1.1,11.8,8.9,11.3,15.7,--,--


In [40]:
# 1. Remover colunas redundantes
cols_to_drop = ["NU_ANO_CENSO", "SG_UF", "NO_REGIAO"]
df_sp = ddf_sp.drop(columns=[c for c in cols_to_drop if c in ddf_sp.columns], errors="ignore")

# 2. Remover linhas totalmente vazias (quase nunca ocorre)
df_sp = df_sp.dropna(how="all")

# 3. Normalizar NO_CATEGORIA e NO_DEPENDENCIA
df_sp["NO_CATEGORIA"] = (
    df_sp["NO_CATEGORIA"].astype(str).str.strip().str.lower()
)

df_sp["NO_DEPENDENCIA"] = (
    df_sp["NO_DEPENDENCIA"].astype(str).str.strip().str.lower()
)

# 4. Filtrar apenas categoria total e dependência total
df_mun = df_sp[
    (df_sp["NO_CATEGORIA"] == "total") &
    (df_sp["NO_DEPENDENCIA"] == "total")
].copy()

# 5. Remover as colunas textuais se quiser
df_mun = df_mun.drop(columns=["NO_CATEGORIA", "NO_DEPENDENCIA"], errors="ignore")

# 6. Remover duplicatas por município (se houver)
df_mun = df_mun.drop_duplicates(subset=["CO_MUNICIPIO"])

# 7. Check final
print("Número de linhas após filtro Total/Total:", df_mun.compute().shape[0])
print("Número de municípios únicos:", df_mun["CO_MUNICIPIO"].compute().nunique())

df_mun.compute().head()


Número de linhas após filtro Total/Total: 645
Número de municípios únicos: 645


,CO_MUNICIPIO,NO_MUNICIPIO,1_CAT_FUN,1_CAT_FUN_AI,1_CAT_FUN_AF,1_CAT_FUN_01,1_CAT_FUN_02,1_CAT_FUN_03,1_CAT_FUN_04,1_CAT_FUN_05,...,3_CAT_FUN_06,3_CAT_FUN_07,3_CAT_FUN_08,3_CAT_FUN_09,3_CAT_MED,3_CAT_MED_01,3_CAT_MED_02,3_CAT_MED_03,3_CAT_MED_04,3_CAT_MED_NS
40231,3500709.0,Agudos,97.2,97.4,96.9,99.4,99.1,91.8,99.4,98,...,1.5,1.1,1.2,4.9,5.2,5.3,5.5,4.8,--,--
40624,3503950.0,Aspásia,99.5,100,98.9,100,100,100,100,100,...,0,0,0,0,2.2,4.8,0,0,--,--
40678,3504404.0,Avanhandava,99.6,100,99.1,100,100,100,100,100,...,1.3,0.5,0.6,1.3,5,3.7,5.9,5.3,--,--
41010,3507209.0,Borá,100,100,100,100,100,100,100,100,...,0,0,0,0,3.3,8.4,0,0,--,--
41249,3509205.0,Cajamar,99.1,99.7,98.5,99.7,99.9,99.5,99.9,99.4,...,0.3,0.1,0.2,0.5,1.5,1.9,1.2,1.1,--,--


In [42]:

# Identificar colunas de taxa (somente estas serão limpas)
tax_cols = [c for c in ddf_sp.columns if c.startswith(("1_", "2_", "3_"))]

# Função de limpeza aplicada em cada partição (Pandas → rápido)
def clean_partition(pdf):
    # valores especiais que queremos substituir por NaN
    special_values = ["--", "...", ".", "-", "—", "", " ", "NA", "N/A"]

    # 1. substituir valores especiais APENAS nas tax_cols
    pdf[tax_cols] = pdf[tax_cols].replace(special_values, np.nan)

    # 2. normalizar vírgula decimal -> ponto
    pdf[tax_cols] = pdf[tax_cols].apply(
        lambda col: col.astype(str).str.replace(",", ".", regex=False)
    )

    # 3. converter para float
    pdf[tax_cols] = pdf[tax_cols].apply(
        lambda col: pd.to_numeric(col, errors="coerce")
    )

    return pdf

# Aplicar limpeza paralelizada
df_sp_clean = ddf_sp.map_partitions(clean_partition)

# Verificar rapidamente (lazy)
df_sp_clean.compute().head()


,NU_ANO_CENSO,NO_REGIAO,SG_UF,CO_MUNICIPIO,NO_MUNICIPIO,NO_CATEGORIA,NO_DEPENDENCIA,1_CAT_FUN,1_CAT_FUN_AI,1_CAT_FUN_AF,...,3_CAT_FUN_06,3_CAT_FUN_07,3_CAT_FUN_08,3_CAT_FUN_09,3_CAT_MED,3_CAT_MED_01,3_CAT_MED_02,3_CAT_MED_03,3_CAT_MED_04,3_CAT_MED_NS
40160,2024,Sudeste,SP,3500105.0,Adamantina,Total,Total,99.5,99.7,99.3,...,0.2,0.5,0.0,0.7,8.6,6.0,8.5,11.9,NaN,NaN
40161,2024,Sudeste,SP,3500105.0,Adamantina,Urbana,Total,99.5,99.7,99.3,...,0.2,0.5,0.0,0.7,9.4,6.7,8.9,12.7,NaN,NaN
40162,2024,Sudeste,SP,3500105.0,Adamantina,Rural,Total,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,NaN,NaN
40163,2024,Sudeste,SP,3500105.0,Adamantina,Total,Estadual,99.4,NaN,99.4,...,0.4,0.8,0.0,1.1,10.7,7.6,10.6,14.4,NaN,NaN
40164,2024,Sudeste,SP,3500105.0,Adamantina,Urbana,Estadual,99.4,NaN,99.4,...,0.4,0.8,0.0,1.1,11.8,8.9,11.3,15.7,NaN,NaN
